Сначала с помощью pip install устанавливаем все нужные нам библиотеки и файлы с выборкой с гитхаба, а также файл с функциями (чтоб не писать их в код)

In [29]:
!pip install stanza

In [30]:
!pip install navec

In [31]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-12-29 01:26:19--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’

navec_hudlit_v1_12B 100%[===================>]  50.56M  15.9MB/s    in 3.4s    

2023-12-29 01:26:23 (14.7 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’ saved [53012480/53012480]



In [47]:
!wget -O train_reviews.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
!wget -O train_aspects.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget -O train_cats.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_cats.txt
!wget https://github.com/Timofeidedov/NLP2024_project/blob/main/functions.py

--2023-12-29 01:59:08--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446118 (436K) [text/plain]
Saving to: ‘train_reviews.txt’

train_reviews.txt   100%[===================>] 435.66K  --.-KB/s    in 0.04s   

2023-12-29 01:59:08 (11.9 MB/s) - ‘train_reviews.txt’ saved [446118/446118]

--2023-12-29 01:59:08--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [48]:
import transformers

from sklearn.model_selection import train_test_split

import pandas as pd

from tqdm import tqdm

import stanza
stanza.download('ru')

import numpy as np

from transformers import BertModel

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer

from torch.nn.utils.rnn import pad_sequence
execfile('/content/functions.py')

INFO:stanza:Downloading default packages for language: ru (Russian) ...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


Здесь делим выборку на train И dev как было сделано в тетрадке с baseline

In [34]:
texts, ids = [], []
with open('/content/train_reviews.txt') as f:
    for line in f:
        text_id, text = line.rstrip('\r\n').split('\t')
        texts.append(text)
        ids.append(text_id)
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)
train_aspects, dev_aspects = [], []
with open('/content/train_aspects.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_aspects.append(line)
        if text_id in dev_ids:
            dev_aspects.append(line)
train_sentiment, dev_sentiment = [], []
with open('/content/train_cats.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_sentiment.append(line)
        if text_id in dev_ids:
            dev_sentiment.append(line)
with open('train_split_aspects.txt', 'w') as f:
    for l in train_aspects:
        print(l, file=f)
with open('dev_aspects.txt', 'w') as f:
    for l in dev_aspects:
        print(l, file=f)
with open('train_split_reviews.txt', 'w') as f:
    for i, l in zip(train_ids, train_texts):
        print(i, l, sep="\t", file=f)
with open('dev_reviews.txt', 'w') as f:
    for i, l in zip(dev_ids, dev_texts):
        print(i, l, sep="\t", file=f)
with open('train_split_cats.txt', 'w') as f:
    for l in train_sentiment:
        print(l, file=f)
with open('dev_cats.txt', 'w') as f:
    for l in dev_sentiment:
        print(l, file=f)

## Выделение аспектной лексики

### Работа с данными
Для выделения аспектов сначала "запомним" те токены, которые были выделены как важные в train выборке. Затем с помощью написанных нами токенизатора и ABTEBertа токенизируем предложения и выделим те токены, которые модель посчитает наиболее важными (на основе данных из train_datasetа)

In [49]:
train_reviews_data = pd.read_csv('train_split_reviews.txt', names=['id', 'review'], delimiter='\t')
train_aspects_data = pd.read_csv('train_split_aspects.txt', names=['id', 'cat', 'token', 'from', 'to', 'sent'], delimiter='\t')
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

sents, positions = [], []

for review in tqdm(train_reviews_data.iterrows()):
    text = review[1]['review']
    rev_sents, rev_positions = get_sentences(text)
    sents.append(rev_sents)
    positions.append(rev_positions)

ys = []
for text in sents:
    ys.append([])
    for sent in text:
        ys[-1].append([])
        for word in sent:
            ys[-1][-1].append(0)

i = 0
for review in train_reviews_data.iterrows():
    for term in train_aspects_data[train_aspects_data['id'] == review[1]['id']].iterrows():
        start, end = term[1]['from'], term[1]['to']
        for j in range(len(sents[i])):
            for k in range(len(sents[i][j])):
                if positions[i][j][k][0] >= start and positions[i][j][k][1] < end:
                    ys[i][j][k] = 1
    i += 1

train_reviews_data['sents_review'] = sents
train_reviews_data['y'] = ys

sents_for_df = []
for review in train_reviews_data.iterrows():
    idx = review[1]['id']
    for i in range(len(review[1]['sents_review'])):
        new_sent = {}
        new_sent['review_id'] = idx
        new_sent['sent'] = review[1]['sents_review'][i]
        if len(new_sent['sent']) > 100:
            new_sent['sent'] = new_sent['sent'][:100]
        new_sent['y'] = review[1]['y'][i]
        sents_for_df.append(new_sent)

sents_df = pd.DataFrame(sents_for_df)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| lemma     | syntagrus_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
213it [00:10, 20.38it/s]


### Модель

In [36]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags = self.df.iloc[idx, 1:].values

        bert_tokens = []
        bert_tags = []
        for i in range(len(tokens)):
            t = self.tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])]*len(t)

        bert_ids = self.tokenizer.convert_tokens_to_ids(bert_tokens)

        ids_tensor = torch.tensor(bert_ids)
        tags_tensor = torch.tensor(bert_tags)
        return bert_tokens, ids_tensor, tags_tensor

    def __len__(self):
        return len(self.df)

In [38]:
class ABTEBert(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(ABTEBert, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 2)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, ids_tensors, tags_tensors, masks_tensors):
        bert_outputs = self.bert(input_ids=ids_tensors, attention_mask=masks_tensors, return_dict=False)
        bert_outputs = bert_outputs[0]

        linear_outputs = self.linear(bert_outputs)
        if tags_tensors is not None:
            tags_tensors = tags_tensors.view(-1)
            linear_outputs = linear_outputs.view(-1,2)
            loss = self.loss_fn(linear_outputs, tags_tensors)
            return loss
        else:
            return linear_outputs

В этой ячейке берем модель, которую будем использовать, и дообучаем ее. Также здесь берем модели для оценки тональности и navec для классификации по близости между векторами, и прописываем функцию для этого. Выбираем "базовые" слова для каждой категории и смотрим, к какому из них ближе всего вектор "нашего" токена. Такую категорию и приписываем.

In [39]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
my_dset = MyDataset(sents_df, tokenizer)
model = ABTEBert("DeepPavlov/rubert-base-cased")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_abte(model, my_dset, 5, device=DEVICE)
from navec import Navec
path = '/content/navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)
ton_model = transformers.pipeline(task="text-classification", model="MonoHime/rubert-base-cased-sentiment-new")
def classify(token):
    try:
        token_vector = navec[token]
        cats = ['Whole', 'Interior', 'Food', 'Service', 'Price']
        base_words = ['ресторан', 'интерьер', 'еда', 'обслуживание', 'цена']
        base_vectors = [navec[word] for word in base_words]
        idx = np.argmin(np.array([np.linalg.norm(token_vector - base_vector) for base_vector in base_vectors]))
        return cats[idx]
    except KeyError:
        return 'Whole'

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


epoch: 0	batch: 1/299	loss: 0.7732300758361816
epoch: 0	batch: 2/299	loss: 0.40606895089149475
epoch: 0	batch: 3/299	loss: 0.20268851518630981
epoch: 0	batch: 4/299	loss: 0.10651712864637375
epoch: 0	batch: 5/299	loss: 0.3461230993270874
epoch: 0	batch: 6/299	loss: 0.10931430011987686
epoch: 0	batch: 7/299	loss: 0.04558185487985611
epoch: 0	batch: 8/299	loss: 0.14440439641475677
epoch: 0	batch: 9/299	loss: 0.21311886608600616
epoch: 0	batch: 10/299	loss: 0.07686159014701843
epoch: 0	batch: 11/299	loss: 0.04530131071805954
epoch: 0	batch: 12/299	loss: 0.10950654745101929
epoch: 0	batch: 13/299	loss: 0.1988222897052765
epoch: 0	batch: 14/299	loss: 0.13794028759002686
epoch: 0	batch: 15/299	loss: 0.005036262795329094
epoch: 0	batch: 16/299	loss: 0.161934033036232
epoch: 0	batch: 17/299	loss: 0.039078302681446075
epoch: 0	batch: 18/299	loss: 0.1216619685292244
epoch: 0	batch: 19/299	loss: 0.03351683169603348
epoch: 0	batch: 20/299	loss: 0.11272701621055603
epoch: 0	batch: 21/299	loss: 0.00

## Собираем все воедино

Здесь применяем к нашему датасету все созданные нами функции, получая в итоге файл с предсказанными аспектами, а также пишем код, который будет смотреть, какая тональность приписана к какой категории. Это сделано примерно также, как в baseline (потому что это один из немногих способов выделять "both")

In [46]:
dev_reviews = pd.read_csv('dev_reviews.txt', names=['id', 'review'], delimiter='\t')
asps_df = absa_pred_df(dev_reviews, tokenizer, model, ton_model, device=DEVICE)
asps_df.to_csv('dev_pred_aspects.txt',header=False, index=False)
cats = ['Food', 'Interior', 'Price', 'Whole', 'Service']
cats_list = []
for i, group in asps_df.groupby(['id']):
    for cat in cats:
        new_dct = {'id': group['id'].values[0],'category' : cat, 'sentiment': 'absent'}
        sentiments = group[group['cat'] == cat].values
        if 'positive' in sentiments and 'negative' in sentiments:
            new_dct['sentiment'] = 'both'
        elif 'positive' in sentiments:
            new_dct['sentiment'] = 'positive'
        elif 'negative' in sentiments:
            new_dct['sentiment'] = 'negative'
        elif 'neutral' in sentiments:
            new_dct['sentiment'] = 'neutral'
        cats_list.append(new_dct)
cats_df = pd.DataFrame(cats_list)
cats_df.head(10)
cats_df.to_csv('dev_pred_cats.txt',header=False, index=False)

71it [01:43,  1.45s/it]
<ipython-input-46-31478bad2ff0>:6: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, group in asps_df.groupby(['id']):
